# Collaborative Filtering Recommender System

## Table of Contents
1. Loading in User Ratings
2. Data Split
3. Baseline Model
4. SVD Model
    - 4.a. SVD Grid Search
5. SVD++ Model
    - 5.a. SVD++ Grid Search
6. NMF Model
    - 6.a. NMF Grid Search
7. Final Model
8. Recommendation Function
9. Conclusion

*Please note that essential project background regarding the business question and data sources are in the introduction of the data preparation notebook

In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor


## 1. Loading in User Ratings

In [3]:
ratings = pd.read_csv('data/gr_reviews_clean.csv')
ratings.head()

,Unnamed: 0,user_id,rating,book_id,review_text
0,0,8842281e1d1347389f2ab93d60773d4d,5,18245960,This is a special book. It started slow for ab...
1,1,8842281e1d1347389f2ab93d60773d4d,3,16981,Recommended by Don Katz. Avail for free in Dec...
2,2,8842281e1d1347389f2ab93d60773d4d,3,28684704,"A fun, fast paced science fiction thriller. I ..."
3,3,8842281e1d1347389f2ab93d60773d4d,0,27161156,Recommended reading to understand what is goin...
4,4,8842281e1d1347389f2ab93d60773d4d,4,25884323,"I really enjoyed this book, and there is a lot..."


In [4]:
ratings.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1373966 entries, 0 to 1373965
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_id           1373966 non-null  object
 1   rating            1373966 non-null  int64 
 2   book_id           1373966 non-null  int64 
 3   review_sentences  1373966 non-null  object
dtypes: int64(2), object(2)
memory usage: 41.9+ MB


## 2. Data Split

In [5]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id','book_id','rating']], reader)

In [6]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=1)

## 3. Baseline Model

Here I'll use root mean squared error (RMSE) as my metric for evaluating the success of my model

In [ ]:
baseline = BaselineOnly()
baseline.fit(trainset)

# Return test predictions for model fit on trainset
predictions = baseline2.test(testset)

# Save RMSE score to variable
baseline_only = accuracy.rmse(predictions)

## 4. SVD Model

In [14]:
svd_simple = SVD(random_state=42)

# Run 5-fold cross-validation
results_svd_simple = cross_validate(svd_simple, data, measures=['RMSE'], cv=5, n_jobs = -1, verbose=True)

# Fit model to trainset to get predictions
svd_simple.fit(trainset)
predictions_svd_simple = svd_simple.test(testset)

# Use predictions to get RMSE score, then save RMSE to variable
svd_simple_acc = accuracy.rmse(predictions_svd_simple)

# Print cross-val and testset results results
print(results_svd_simple)
print(svd_simple_acc)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1167  1.1109  1.1146  1.1168  1.1105  1.1139  0.0027  
Fit time          20.49   20.10   19.84   19.40   18.76   19.72   0.60    
Test time         3.78    3.70    2.94    2.63    1.82    2.97    0.73    
RMSE: 1.1157
{'test_rmse': array([1.11673995, 1.11091317, 1.11459762, 1.11680213, 1.11054797]), 'fit_time': (20.49123787879944, 20.09840202331543, 19.836138010025024, 19.40072011947632, 18.75564980506897), 'test_time': (3.77850604057312, 3.6963131427764893, 2.938753843307495, 2.6298141479492188, 1.8161752223968506)}
1.115713059999836


### 4.a. SVD Grid Search

In [16]:
svd_grid = {'n_factors':[20, 100],
            'n_epochs': [5, 10], 
            'lr_all': [0.002, 0.005],
            'reg_all': [0.4, 0.6],
           'biased': [True, False]}
gs1_svd = GridSearchCV(SVD,param_grid=svd_grid,n_jobs = -1,joblib_verbose=5, cv=3)
gs1_svd.fit(data)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done  96 out of  96 | elapsed:  4.5min finished


NameError: name 'gs1_svd_' is not defined

In [17]:
gs1_svd.best_params['rmse']


{'n_factors': 20,
 'n_epochs': 10,
 'lr_all': 0.005,
 'reg_all': 0.4,
 'biased': True}

In [20]:
gs1_svd_best = SVD(n_factors=20, n_epochs=10, lr_all=0.005, reg_all=0.4, biased=True)
gs1_svd_best.fit(trainset)
predictions = gs1_svd_best.test(testset)
print(accuracy.rmse(predictions))

RMSE: 1.1056
1.1056224350578843


## 5. SVD++ Model

In [ ]:
svd_plus = SVDpp(random_state=42)
results_svd_plus = cross_validate(svd_plus, data, measures=['RMSE'], cv=5, n_jobs = -1, verbose=True)
svd_plus.fit(trainset)
predictions_svd_plus = svd_plus.test(testset)
svd_plus_acc = accuracy.rmse(predictions)
print(results)
print(svd_plus_acc)

### 5.a. SVD++ Grid Search

## 6. NMF Model

### 6.a. NMF Grid Search

## 7. Final Model

## 8. Recommendation Function

## 9. Conclusion